In [17]:
'''Trains a simple convnet on the face landmark dataset.
Adapted from Keras MNIST CNN example code.
'''

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils


batch_size = 10
nb_landmarks = 5
nb_epoch = 2
nb_verbose = 1
#nb_validation_data = (X_test, _Y_test)
nb_validation_data = None

# input image dimensions
img_chns, img_rows, img_cols = 3, 20, 20
# number of convolutional filters to use
nb_filters = [32, 64]
# size of pooling area for max pooling
nb_pool_sizes = [(2, 2), (2, 2)]
# convolution kernel size
nb_conv = 3
# activator
nb_activator = 'tanh'
# number of fully connected neurons in the penultimate layer
nb_penu_neurons = 128
# size of output vector, four "corner values" for each landmark
nb_output_size = nb_landmarks * 4

input_img = Input(shape=(img_chns, img_rows, img_cols), dtype='float32', name='input_img')

x = input_img
x = (Convolution2D(nb_filters[0], nb_conv, nb_conv))(x)
x = (Activation(nb_activator))(x)
x = (MaxPooling2D(pool_size=nb_pool_sizes[0]))(x)
x = (Convolution2D(nb_filters[1], nb_conv, nb_conv))(x)
x = (Activation(nb_activator))(x)
x = (MaxPooling2D(pool_size=nb_pool_sizes[1]))(x)
x = (Dropout(0.25))(x)

x = (Flatten())(x)
x = (Dense(nb_penu_neurons))(x)
x = (Activation(nb_activator))(x)
x = (Dropout(0.5))(x)
#output_landmarks = [Activation('sigmoid')((Dense(4, name='landmark-%d'%(_)))(x)) for _ in range(nb_landmarks)]
output = Activation('sigmoid')(Dense(1, name="detect_face")(x))

model = Model(input=input_img, output=output)

model.compile(loss='binary_crossentropy',
              #loss_weights=[1, 1, 5, 1, 1],
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=nb_verbose, validation_data=nb_validation_data)

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
15528/15528 [==============================] - 17s - loss: 0.1474 - acc: 0.9444    
Epoch 2/2
15528/15528 [==============================] - 18s - loss: 0.0711 - acc: 0.9740    
Test score: 0.0600529295538
Test accuracy: 0.979649665121


In [ ]:
# save model
model_filename = '../model/corners-all(ratio-0.8,rand-1337),20px,500ep'
print(model.to_json(), file=open(model_filename+'.json', 'w'))
model.save_weights(model_filename+'.weight')

In [18]:
Y_pred = model.predict(X_test)

In [19]:
# observe learning results
import random
y_converter = lambda y : np.array(list(map(corners_to_coord, y)))

plt.figure()
ind_picked = list(random.sample(range(len(Y_test)), 30))
for iplt, i in enumerate(ind_picked) :
    plt.subplot(5, 6, iplt+1)
    img = X_test[i].transpose((1,2,0))
    print(i, iplt, Y_test[i], Y_pred[i][0], int(Y_test[i])==int(Y_pred[i][0]+.5))
    io.imshow(img)
    '''
    pts2 = y_converter(Y_test[i])
    plt.plot(pts2[:,0]*img_rows, pts2[:,1]*img_cols, 'o')
    pts = y_converter(Y_pred[i])
    plt.plot(pts[:,0]*img_rows, pts[:,1]*img_cols, 'ro')
    '''
    plt.axis('off')
plt.tight_layout()
io.show()

2400 0 0.0 0.000102047997643 True
547 1 1.0 0.999546229839 True
2214 2 1.0 0.997172415257 True
1244 3 1.0 0.999528050423 True
2462 4 0.0 0.000159538467415 True
3735 5 1.0 0.999836802483 True
1472 6 1.0 0.999602735043 True
2372 7 0.0 9.17866636883e-05 True
1880 8 1.0 0.999778449535 True
1066 9 0.0 0.00053214840591 True
2987 10 1.0 0.999618172646 True
2916 11 1.0 0.99924904108 True
49 12 0.0 0.0647002160549 True
622 13 0.0 0.00011810286378 True
1844 14 1.0 0.997212707996 True
3382 15 1.0 0.998977184296 True
1084 16 0.0 0.0454299002886 True
1140 17 0.0 0.000623564526904 True
2861 18 1.0 0.972686827183 True
2109 19 0.0 9.64103601291e-05 True


C:\Anaconda3\envs\cnn\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:74: UserWarning: Low image dynamic range; displaying image with stretched contrast.
  warn("Low image dynamic range; displaying image with "


1393 20 0.0 0.000121594042866 True
3579 21 1.0 0.999157547951 True
2827 22 0.0 0.0003537038574 True
1997 23 1.0 0.989103078842 True
1547 24 0.0 0.00019816114218 True
1767 25 0.0 0.960856497288 False
2529 26 0.0 0.000507330638357 True
352 27 0.0 0.992663502693 False
720 28 1.0 0.999788701534 True
1752 29 1.0 0.999783813953 True


In [ ]:
def Y_data_to_model(Y) :
    return list(np.transpose(Y, (1,0,2)))
def Y_model_to_data(Y) :
    return np.transpose(np.array(Y), (1,0,2))

In [1]:
from face_data import *
_X = X; _Y = Y;

X shape: (8003, 3, 20, 20)
Y shape: (8003, 5, 2)


In [5]:
(X_train, Y_train), (X_test, Y_test) = split_data(X, Y, ratio_train=0.8, rand_seed=1337)

X_train shape: (15528, 3, 20, 20)
Y_train shape: (15528,)
15528 train samples
3882 test samples


In [2]:
# non face data
from glob import glob
paths = glob('../../../nonface_20/*/*.png')

from skimage import data, io, filters
import numpy as np
X_nonface = np.array([np.transpose(io.imread(f), (2,0,1)) for f in paths if np.amax(io.imread(f)) > 1])
X_nonface.astype('float32')
X_nonface = X_nonface/255

In [3]:
Y = np.append(np.ones((_X.shape[0], 1)), np.zeros((X_nonface.shape[0], 1)))
X = np.append(_X, X_nonface, axis=0)

In [ ]:
[f for f in paths if np.amax(io.imread(f)) <= 1]

In [ ]:

np.amax(X)

In [15]:
plt.figure()
io.imshow(X_test[2907].transpose((1,2,0)))
io.show()

C:\Anaconda3\envs\cnn\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:74: UserWarning: Low image dynamic range; displaying image with stretched contrast.
  warn("Low image dynamic range; displaying image with "
